In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
import datetime as dt

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import gc

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans


import folium
import geopandas as gpd
from haversine import haversine

import optuna
import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)


bold = ['\033[1m', '\033[0m']
seed = 228

In [18]:
train = pd.read_csv('train.csv') # index_col=0 is the id column
test = pd.read_csv('test.csv') # index_col=0 is the id column
submission = pd.read_csv('sample_submission.csv')
y = train['emission']

## 1. ID Generation:
The code first generates a unique ID for each row by combining latitude and longitude. It extracts digits from both coordinates, concatenates them, and then maps them to a new unique ID.

In [20]:
def get_id(row):
    return int(''.join(filter(str.isdigit, str(row['latitude']))) + ''.join(filter(str.isdigit, str(row['longitude']))))

train['id'] = train[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
test['id'] = test[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
new_ids = {id_: new_id for new_id, id_ in enumerate(train['id'].unique())}
train['id'] = train['id'].map(new_ids)
test['id'] = test['id'].map(new_ids)

## 2. Extract Month from Year and Week:
A function to extract the month given a year and week number is defined.

In [21]:
def get_month(row):
    date = dt.datetime.strptime(f'{row["year"]}-{row["week_no"]+1}-1', "%Y-%W-%w")
    return date.month


## 3. Define Geographic Points of Interest:
Several important geographic locations are defined for later distance computations.

In [22]:
rwanda_center = (-1.9607, 29.9707)
park_biega = (-1.8866, 28.4518)
kirumba = (-0.5658, 29.1714)
massif = (-3.42, 28.592)
lake = (-2.0073, 31.6269)

## 4. Compute Distances to Cluster Centers:
The code computes distances from given points to several cluster centers.

In [23]:
def cluster_features(df, cluster_centers):
    for i, cc in enumerate(cluster_centers.values()):
        df[f'cluster_{i}'] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), cc, unit='ft'), axis=1)
    return df

## 5. Data Preprocessing:
This section is the bulk of the code. It involves:

 1. Filtering columns
 2. Forward and backward filling missing values
 2. Creating several lag and rotational features
 4. Calculating distances to defined geographic locations
 5. Extracting month and identifying periods of COVID-19 and lockdowns

In [24]:
def preprocessing(df):
    # Select important columns
   
    cols_save = ['id', 'latitude', 'longitude', 'year', 'week_no', 'Ozone_solar_azimuth_angle']
    df = df[cols_save]
    
    # Fill missing values for 'Ozone_solar_azimuth_angle'
    good_col = 'Ozone_solar_azimuth_angle'
    df[good_col] = df.groupby(['id', 'year'])[good_col].ffill().bfill()
    df[f'{good_col}_lag_1'] = df.groupby(['id', 'year'])[good_col].shift(1).fillna(0)
    
    # Compute rotated features for latitude and longitude
    df['rot_15_x'] = (np.cos(np.radians(15)) * df['longitude']) + \
                     (np.sin(np.radians(15)) * df['latitude'])
    
    df['rot_15_y'] = (np.cos(np.radians(15)) * df['latitude']) + \
                     (np.sin(np.radians(15)) * df['longitude'])

    df['rot_30_x'] = (np.cos(np.radians(30)) * df['longitude']) + \
                     (np.sin(np.radians(30)) * df['latitude'])

    df['rot_30_y'] = (np.cos(np.radians(30)) * df['latitude']) + \
                     (np.sin(np.radians(30)) * df['longitude'])
    
    # Calculate distances to specified geographic locations
    for col, coors in zip(
        ['dist_rwanda', 'dist_park', 'dist_kirumba', 'dist_massif', 'dist_lake'], 
        [rwanda_center, park_biega, kirumba, massif, lake]
    ):
        df[col] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), coors, unit='ft'), axis=1)
    
    # Extract month and label periods of COVID-19 and lockdowns
    df['month'] = df[['year', 'week_no']].apply(lambda row: get_month(row), axis=1)
    df['is_covid'] = (df['year'] == 2020) & (df['month'] > 2) | (df['year'] == 2021) & (df['month'] == 1)
    df['is_lockdown'] = (df['year'] == 2020) & ((df['month'].isin([3,4])))
    
    df.fillna(0, inplace=True)
    return df

train = preprocessing(train)
test = preprocessing(test)


## 6. Clustering:
Based on latitude and longitude, KMeans clustering is applied to group locations into 12 clusters

In [26]:
df = pd.concat([train, test], axis=0, ignore_index=True)
coordinates = df[['latitude', 'longitude']].values
clustering = KMeans(n_clusters=25, max_iter=100, random_state=seed).fit(coordinates)
cluster_centers = {i: tuple(centroid) for i, centroid in enumerate(clustering.cluster_centers_)}
df = cluster_features(df, cluster_centers)

## 7. Splitting and Cleaning:
Finally, the preprocessed dataframe is split back into train and test, and unused variables are removed.

In [27]:

train = df.iloc[:-len(test),:]
test = df.iloc[-len(test):,:]
del df

X = train.drop('id', axis=1)
test = test.drop('id', axis=1)
test=test.reset_index(drop=True)

In [34]:
import optuna
import xgboost as xgb
from optuna.integration import XGBoostPruningCallback

dtrain = xgb.DMatrix(X, label=y)

def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-5, 1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 1, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'eta': trial.suggest_float('eta', 1e-5, 1, log=True),
        'gamma': trial.suggest_float('gamma', 0, 0.4)

    }
    
    pruning_callback = XGBoostPruningCallback(trial, 'test-rmse')
    cv_result = xgb.cv(param, dtrain, num_boost_round=5000, nfold=3, callbacks=[pruning_callback], verbose_eval=False)
    
    return cv_result["test-rmse-mean"].values[-1]

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2023-08-06 14:44:14,002] A new study created in memory with name: no-name-44f8eae4-9a0b-4a0d-b78a-d25b507a5b4f


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
logo = LeaveOneGroupOut()

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

xgb_params = {
    'max_depth': 6,
    'eta': 0.01,
    'colsample_bytree': 0.66,
    'subsample': 0.76,
    'min_child_weight': 2,
    'lambda': 1, 
    'gamma': 1,
    
    # 'tree_method': 'gpu_hist',
    'booster': 'gbtree',
    # 'predictor':'gpu_predictor',
    'seed': seed,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

scores = []
for fold, (train_idx, val_idx) in enumerate(logo.split(X, y, X['year'])):

    dtrain = xgb.DMatrix(
        X.iloc[train_idx], 
        label=y.iloc[train_idx]
    )
    dvalid = xgb.DMatrix(
        X.iloc[val_idx], 
        label=y.iloc[val_idx]
    )

    model = xgb.train(
        params=xgb_params, 
        dtrain=dtrain, 
        num_boost_round=5000,
        evals=[(dvalid, 'eval')], 
        verbose_eval=False,
        callbacks=[
            xgb.callback.EarlyStopping(
                rounds=500,
                data_name='eval',
                maximize=False,
                save_best=True
            )
        ]
    )
    
    val_preds = model.predict(dvalid)
    val_score = rmse(y.iloc[val_idx], val_preds)
    scores.append(val_score)
    print(f'FOLD {fold+1} RMSE: {bold[0]}{round(val_score, 6)}{bold[1]}')
    
    del dtrain, dvalid, val_preds, val_score, model
    gc.collect()

print(f'\nCV RMSE: {bold[0]}{round(np.mean(scores), 6)}{bold[1]}')

FOLD 1 RMSE: 17.283086
FOLD 2 RMSE: 23.55203
FOLD 3 RMSE: 19.292354

CV RMSE: 20.04249
